# Multiprocessing: Exec; inter-process communication
_COSC 208, Introduction to Computer Systems, 2023-11-08_

## Announcements
* Project 3 due tomorrow @ 11pm

    _You **may not share or copy code/answers from other students.** Violations of this policy will be considered facilitating academic misconduct and/or cheating according to Colgate’s academic honor code._
    
    _Additionally, you **must clearly indicate if any code is copied from or inspired by external sources** (e.g., Stack Overflow, ChatGPT, GitHub Copilot) and adhere to the licensing requirements of such code. Violations of this policy will be considered cheating or plagiarism according to Colgate’s academic honor code. Note that question and answer forums (e.g., Stack Overflow) are often unreliable, and AI-based tools (e.g., ChatGPT) have significant limitations and legal complications._
    
    _**If you are struggling to complete an activity—due to a lack of understanding, time pressure, personal matters, etc.—please contact me. I want you to succeed, and I will do everything I can to help you.** Violating the academic honesty expectations for the course (almost) always leads to a bad experience for you and me. If you are unsure what constitutes academic misconduct, please contact me as soon as possible._

## Outline
* Warm-up
* Running a different program
* Interprocess communication

## Warm-up

* Q1: _What are all possible outputs of this program, assuming the process identifier (PID) of the parent process is 50 and the PID of the child process is 60?_

In [ ]:
#include <stdio.h>
#include <unistd.h>    
int main() {
    int n = 20;
    int x = fork();
    if (x < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (x == 0) {
        n += 5;
    } else {
        wait(NULL);
        n += 10;
    }
    printf("x=%d n=%d\n", x, n);
}

    x=0 n=25
    x=60 n=30

<div style="height:10em;"></div>

🛑 **STOP here** after completing the above question; if you have extra time **take a few deep breaths** to reduce stress.

<div style="page-break-after:always;"></div>

## Running a different program

* `int execv(const char *path, const char *argv[])`
    * Used to switch which program is running in a process — replaces current code with code for a new program and starts executing that program from main
    * `path` == full path to program
    * `argv` == array of strings containing the full path to the program, any command line arguments, and `NULL`

* Example program

In [5]:
#include <stdio.h>
#include <unistd.h>
int main(int argc, char **argv) {
    printf("Begin\n");
    int pid = fork();
    if (pid == 0) {
        printf("Child\n");
        char *cmd[] = { "/bin/date", NULL };
        execv(cmd[0], cmd);
    } else {
        printf("Parent\n");
    }
    printf("End\n");
    return 0;
}

Begin
Parent
End
Child
Sun Nov  5 20:18:17 EST 2023


* Q2: _What is the output produced by running `./progA`, assuming no errors occur?_

In [ ]:
/** progA **/
#include <stdio.h>
#include <unistd.h>
int main() {
    pid_t a = fork();
    if (a == 0) {
        char *cmd[] = { "./progB", NULL };
        execv(cmd[0], cmd);
        printf("A 2nd gen\n");
        return 0;
    } else {
        wait(NULL);
        printf("A 1st gen\n");
        return 0;
    }
}

In [ ]:
/** progB **/
#include <stdio.h>
#include <unistd.h>
int main() {
    pid_t b = fork();
    if (b == 0) {
        printf("B 2nd gen\n");
        return 0;
    } else {
        wait(NULL);
        printf("B 1st gen\n");
        return 0;
    }
}

    B 2nd gen
    B 1st gen
    A 1st gen

<div style="height:2em;"></div>

🛑 **STOP here** after completing the above question; if you have extra time **take a few deep breaths** to reduce stress.

<div style="page-break-after:always;"></div>

## Interprocess communication

* Signals
    * Send using the `kill` system call
    * Register a signal handler using the `signal` system call – must register before calling fork

* Example with signals

In [28]:
#include <signal.h>
#include <stdio.h>
#include <unistd.h>

void signal_handler(int sig) {
    printf("%d received %d\n", getpid(), sig);
}

int main() {
    int result = fork();
    if (result < 0) {
        return 1;
    } else if (result == 0) {
        printf("Child %d\n", getpid());
        if (signal(SIGUSR1, signal_handler) == SIG_ERR) {
            return 1;
        }
        for (int i = 0; i < 1000*100000; i++); // Do work
    } else {
        printf("Parent %d\n", getpid());
        for (int i = 0; i < 1000*1000; i++); // Do work
        if (kill(result, SIGUSR1) < 0) {
            return 1;
        }
        wait(NULL);
    }
    return 0;
}

Parent 3946
Child 3947
3947 received 30


* Message passing
    * pipe – one-way communcation channel for two processes on the same machine
    * socket - dual-way communication channel for two processes on the same or different machines

<div style="page-break-after:always;"></div>

* Example with pipes

In [1]:
#include <signal.h>
#include <stdio.h>
#include <unistd.h>

int main() {
    int pipe_descriptors[2];
    if (pipe(pipe_descriptors) < 0) {
        return 1;
    }

    int result = fork();
    if (result < 0) {
        return 1;
    } else if (result == 0) {
        printf("Child\n");
        char msg[1024];
        read(pipe_descriptors[0], msg, 1024);
        printf("%s\n", msg);
    } else {
        char msg[1024] = "Hello from your parent!";
        write(pipe_descriptors[1], msg, 1024);
        wait(NULL);
        printf("Parent\n");
    }
    return 0;
}

Child
Hello from your parent!
Parent


<div style="page-break-after:always;"></div>

* Q3: _What does the following program output assuming the process identifier (PID) of the parent process is 3995 and the PID of the child process is 3996? (Note: `getpid` gets the PID of the calling process, and `getppid` gets the PID of the calling process's parent process.)_

In [3]:
#include <signal.h>
#include <stdio.h>
#include <unistd.h>

void signal_handler(int sig) {
    if (sig == SIGUSR1) {
        printf("SIGUSR1 %d\n", getpid());
    } else if (sig == SIGUSR2) {
        printf("SIGUSR2 %d\n", getpid());
    }
}

int main() {
    if (signal(SIGUSR1, signal_handler) == SIG_ERR) {
        return 1;
    }
    if (signal(SIGUSR2, signal_handler) == SIG_ERR) {
        return 1;
    }
    
    int x = fork();
    if (x < 0) {
        return 1;
    } else if (x > 0) {
        printf("ElseIf %d\n", getpid());
        for (int i = 0; i < 1000*1000; i++); // Do work
        if (kill(x, SIGUSR1) < 0) {
            return 1;
        }
        wait(NULL);
    } else {
        printf("Else %d\n", getpid());
        for (int i = 0; i < 1000*1000; i++); // Do work
        if (kill(getppid(), SIGUSR2) < 0) {
            return 1;
        }
        for (int i = 0; i < 1000*1000; i++); // Do work
    } 
    return 0;
}

ElseIf 3995
Else 3996
SIGUSR1 3996
SIGUSR2 3995
